<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mahotas

In [1]:
import os
import numpy as np
import nibabel as nib
from zipfile import ZipFile
import csv
import mahotas as mh
import tensorflow as tf
import time

In [6]:
#Se descomprime el archivo con los datos preprocesados
!unzip PruebaDatos.zip #En verdad no es este archivo.
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Archive:  PruebaDatos.zip
   creating: PruebaDatos/
   creating: PruebaDatos/func/
 extracting: PruebaDatos/func/sub-01_task-LTM_run-1_bold.nii.gz  
  inflating: PruebaDatos/func/sub-01_task-LTM_run-1_events.tsv  
Found GPU at: /device:GPU:0


In [0]:
pathArchivoDatosPreprocesados = "PruebaDatos" #Ruta del archivo raiz de los datos preprocesados

sub = [0, 0]
run = 1

def darNombreArchivoImagenes(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

def darNombreArchivoEtiquetas(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

def obtenerImagenes(pImagenes):
  
  imagenes = np.zeros((64,64,30,30))

  for indice in range(0,15):
    for i in range(0,64):
      for j in range(0,64):
          for k in range(0,30):
              imagenes[i][j][k][2*indice] = pImagenes[i][j][k][1+indice*13]
              imagenes[i][j][k][2*indice+1] = pImagenes[i][j][k][2+indice*13]

  return imagenes

#object -> 0
#place -> 1
#face -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        etiquetas.append(0)
        etiquetas.append(0)
      elif(etiqueta == 'place'):

        etiquetas.append(1)
        etiquetas.append(1)
      elif(etiqueta == 'face'):

        etiquetas.append(2)
        etiquetas.append(2)
      else:
        print("Error en el formato.")


  return etiquetas

def importarDatos():
  datosPreprocesados = np.zeros((64,64,30,1800))
  etiquetasPreprocesados = np.zeros(1800)

  contador = 0
  for s1 in range(1,2):#(1,11)

    if(s1 == 10):
      sub[0] = 1
      sub[1] = 0
    else:
      sub[1] = s1

      for r in range(1,2):#(1,7)
        run = r
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,30):
          for i in range(0,64):
            for j in range(0,64):
                for k in range(0,30):
                  datosPreprocesados[i][j][k][30*contador+indice] = muestras[i][j][k][indice]
                  etiquetasPreprocesados[30*contador+indice] = etiquetas[indice]

        contador+=1

    return datosPreprocesados, etiquetasPreprocesados

In [3]:
print("Importando datos...")
test, testet=importarDatos()

Importando datos...


In [0]:
escala = 256

test2 = test[:,:,:,1]
test2 = (test2/test2.max())*escala
test2 = test2.astype(int)
#test2 = test2-1

start = time.time()
print("Calculando descriptores...")
descriptor = mh.features.haralick(test2, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

print(descriptor)
print("Tiempo: {}".format(time.time()-start))

Calculando descriptores...
